In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-06 00:50:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.22MB/s    in 0.2s    

2022-10-06 00:50:20 (6.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [4]:
### Load Amazon Data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"]).show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [6]:
# Filter Pet data frame on total_vote greater than or equal to 20
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [7]:
# Filtr Pet data frame on helpful_votes greater than or equal to 50%
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=0.50")
helpful_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [8]:
# Create a DataFrame or table where there is a Vine review
vine_review_df = helpful_50_df.filter(helpful_50_df["vine"]=="Y")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44759540|R3A71VR1JZD8WF|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          2|           27|         30|   Y|                N|Great idea, not s...|I do not hate thi...| 2015-08-29|
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|    

In [9]:
# Create a DataFrame or table where there isn’t a Vine review
notvine_review_df = helpful_50_df.filter(helpful_50_df["vine"]=="N")
notvine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [10]:
# Calculate the total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews for all Vine and non-Vine reviews
from pyspark.sql.functions import count
# Number of reviews
vine_review_count = vine_review_df.count()
print("Total Number of paid reviews: %f" % vine_review_count)

notvine_review_count = notvine_review_df.count()
print("Total Number of no-paid reviews: %f" % notvine_review_count)

Total Number of paid reviews: 170.000000
Total Number of no-paid reviews: 37840.000000


In [11]:
# Number of 5-star reviews
star5_vine = vine_review_df. filter(vine_review_df["star_rating"]== "5")
star5_vine.show()
print("Total number of 5-star paid reviews : %f" % star5_vine.count())

star5_novine = notvine_review_df. filter(notvine_review_df["star_rating"]== "5")
star5_novine.show()
print("Total number of 5-star no-paid reviews : %f" % star5_novine.count())

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|           72|         72|   Y|                N|My cats went craz...|My cats response ...| 2015-08-29|
|         US|   51817707|R3TS8ZP2FHQ9XR|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          5|    

In [12]:
# Percentage 5-star reviews 
star5_percentage = (star5_vine.count()/vine_review_count)
print("Percentage of 5-star paid reviews : %f" % star5_percentage)

star5_percentage = (star5_novine.count()/notvine_review_count)
print("Percentage of 5-star no-paid reviews : %f"% star5_percentage)


Percentage of 5-star paid reviews : 0.382353
Percentage of 5-star no-paid reviews : 0.544715


In [14]:
#Rating summary 
# Vine reviews
summary_vine_ratings = vine_review_df.groupby("star_rating").count()
summary_vine_ratings.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|    6|
|          3|   27|
|          5|   65|
|          4|   56|
|          2|   16|
+-----------+-----+



In [15]:
#Non-Vine reviews
summary_novine_ratings = notvine_review_df.groupby("star_rating").count()
summary_novine_ratings.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 7569|
|          3| 2711|
|          5|20612|
|          4| 4900|
|          2| 2048|
+-----------+-----+

